In [4]:
import numpy as np

In [79]:
%load_ext autoreload
%autoreload 2
from fedimpute.simulator import Simulator

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [84]:
data = np.random.rand(10000, 10)
data_config = {
    'task_type': 'regression',
    'num_cols': 9,
}

simulator = Simulator()
simulation_results = simulator.simulate_scenario(data, data_config, num_clients = 10)

2024-06-13 04:11:07.014 | DEBUG    | fedimpute.utils.reproduce_utils:setup_clients_seed:29 - [6077  577 7231 5504 5705 2532 6950  956 6523 5134]
2024-06-13 04:11:07.097 | DEBUG    | fedimpute.simulator.data_partition.utils:calculate_data_partition_stats:75 - Client 0	 Size of data: 900	 Labels: 
2024-06-13 04:11:07.097 | DEBUG    | fedimpute.simulator.data_partition.utils:calculate_data_partition_stats:79 - 		 Samples of labels: 
2024-06-13 04:11:07.097 | DEBUG    | fedimpute.simulator.data_partition.utils:calculate_data_partition_stats:80 - --------------------------------------------------
2024-06-13 04:11:07.098 | DEBUG    | fedimpute.simulator.data_partition.utils:calculate_data_partition_stats:75 - Client 1	 Size of data: 900	 Labels: 
2024-06-13 04:11:07.098 | DEBUG    | fedimpute.simulator.data_partition.utils:calculate_data_partition_stats:79 - 		 Samples of labels: 
2024-06-13 04:11:07.099 | DEBUG    | fedimpute.simulator.data_partition.utils:calculate_data_partition_stats:80 

In [85]:
simulation_results.keys()

dict_keys(['clients_train_data', 'clients_test_data', 'clients_train_data_ms', 'clients_seeds', 'global_test_data'])

In [90]:
%load_ext autoreload
%autoreload 2
from fedimpute.execution_environment import FedImputeEnv

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
env = FedImputeEnv()
env.configuration(imputer = 'gain', fed_strategy='fedavg', fit_mode = 'fed')
env.setup(
    clients_train_data=simulation_results['clients_train_data'],
    clients_train_data_ms=simulation_results['clients_train_data_ms'],
    clients_test_data=simulation_results['clients_test_data'],
    clients_seeds=simulation_results['clients_seeds'],
    data_config=data_config,
)

2024-06-13 04:16:13.093 | DEBUG    | fedimpute.execution_environment.client.client:profile:228 - ------------------------------------------------------------------------------------------------------------------------
2024-06-13 04:16:13.093 | DEBUG    | fedimpute.execution_environment.client.client:profile:229 - | Client  0 | DS: (900, 9) | MissDS: (900, 9) | MaskDS: (900, 9) | ImputeDS: (900, 9) | MissRatio: 0.47 |
2024-06-13 04:16:13.094 | DEBUG    | fedimpute.execution_environment.client.client:profile:237 - | MissRatio Cols: [0.53 0.74 0.42 0.63 0.32 0.42 0.42 0.74 0.53] |
2024-06-13 04:16:13.097 | DEBUG    | fedimpute.execution_environment.client.client:profile:228 - ------------------------------------------------------------------------------------------------------------------------
2024-06-13 04:16:13.098 | DEBUG    | fedimpute.execution_environment.client.client:profile:229 - | Client  1 | DS: (900, 9) | MissDS: (900, 9) | MaskDS: (900, 9) | ImputeDS: (900, 9) | MissRatio: 0

In [70]:
print(env.imputer_name)
print(env.fed_strategy_name)

gain
fedavg
